In [360]:
import time
from numba import cuda, float64, jit
import numpy as np
import math
from decimal import *

In [361]:
MATRIX_SIZE = 300
EPSILON = 0.0000000000000000001

In [362]:
def generateMagicSquare():   
    arr = [[(MATRIX_SIZE*y)+x+1 for x in range(MATRIX_SIZE)]for y in range(MATRIX_SIZE)]
 
    for i in range(0,MATRIX_SIZE//4):
        for j in range(0,MATRIX_SIZE//4):
            arr[i][j] = (MATRIX_SIZE*MATRIX_SIZE + 1) - arr[i][j];
     
    for i in range(0,MATRIX_SIZE//4):
        for j in range(3 * (MATRIX_SIZE//4),MATRIX_SIZE):
            arr[i][j] = (MATRIX_SIZE*MATRIX_SIZE + 1) - arr[i][j];
 
    for i in range(3 * (MATRIX_SIZE//4),MATRIX_SIZE):
        for j in range(0,MATRIX_SIZE//4):
            arr[i][j] = (MATRIX_SIZE*MATRIX_SIZE + 1) - arr[i][j];
     
    for i in range(3 * (MATRIX_SIZE//4),MATRIX_SIZE):
        for j in range(3 * (MATRIX_SIZE//4),MATRIX_SIZE):
            arr[i][j] = (MATRIX_SIZE*MATRIX_SIZE + 1) - arr[i][j];
             
    for i in range(MATRIX_SIZE//4,3 * (MATRIX_SIZE//4)):
        for j in range(MATRIX_SIZE//4,3 * (MATRIX_SIZE//4)):
            arr[i][j] = (MATRIX_SIZE*MATRIX_SIZE + 1) - arr[i][j];
    
    return arr 

In [363]:
magicSquare = generateMagicSquare()
print("Generated magic square : ", magicSquare)

# magicSquare = [[15,1,7,9],[4,6,12,14],[5,11,13,3],[10,16,2,8]]
# print("User input magic square : ", magicSquare)

numOfElements = MATRIX_SIZE * MATRIX_SIZE
print("Number of elements : ", numOfElements)

magicSum = int(MATRIX_SIZE * (MATRIX_SIZE * MATRIX_SIZE + 1) / 2)
print("Magic sum : ", magicSum)

multipliedMatrix = []
for i in range(0, MATRIX_SIZE):
    row = []
    for j in range(0, MATRIX_SIZE):
        row.append(magicSquare[i][j] / magicSum)
    multipliedMatrix.append(row)

print("Multiplied matrix : ", multipliedMatrix)

Generated magic square :  [[90000, 89999, 89998, 89997, 89996, 89995, 89994, 89993, 89992, 89991, 89990, 89989, 89988, 89987, 89986, 89985, 89984, 89983, 89982, 89981, 89980, 89979, 89978, 89977, 89976, 89975, 89974, 89973, 89972, 89971, 89970, 89969, 89968, 89967, 89966, 89965, 89964, 89963, 89962, 89961, 89960, 89959, 89958, 89957, 89956, 89955, 89954, 89953, 89952, 89951, 89950, 89949, 89948, 89947, 89946, 89945, 89944, 89943, 89942, 89941, 89940, 89939, 89938, 89937, 89936, 89935, 89934, 89933, 89932, 89931, 89930, 89929, 89928, 89927, 89926, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 1

In [364]:
resultMatrix = np.zeros([MATRIX_SIZE, MATRIX_SIZE])
multipliedMatrix = np.array(multipliedMatrix)

TPB = 4
threadsperblock = (TPB, TPB)
blockspergrid_x = math.ceil(resultMatrix.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(resultMatrix.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

print(threadsperblock)
print(blockspergrid)

@cuda.jit
def fast_matmul(A, B, C):
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float64)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float64)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x

    if x >= C.shape[0] and y >= C.shape[1]:
        return
    
    tmp = 0.
    for i in range(bpg):
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        cuda.syncthreads()

        for j in range(TPB):
            tmp += np.float64(sA[tx, j]) * np.float64(sB[j, ty])

        cuda.syncthreads()

    C[x, y] = tmp

(4, 4)
(75, 75)


In [365]:
checker = 0
iteration = 1

start = time.time()

while checker == 0:
    counter = 0
    multipliedMatrixDev = cuda.to_device(multipliedMatrix)
    resultMatrixDev = cuda.to_device(resultMatrix)
    
    fast_matmul[blockspergrid, threadsperblock](multipliedMatrixDev, multipliedMatrixDev, resultMatrixDev)
    
    resultMatrix = resultMatrixDev.copy_to_host()
    multipliedMatrix = multipliedMatrixDev.copy_to_host()
    
    for i in range(MATRIX_SIZE):
        for j in range(MATRIX_SIZE):
            if (abs(np.float64(multipliedMatrix[i][j]) - np.float64(resultMatrix[i][j]) < np.float64(EPSILON))):
                counter += 1
    
    if (counter == numOfElements):
        checker = 1
        end = time.time()
    
    iteration += 1
    multipliedMatrix = np.float64(resultMatrix)
    

print("Iteration : " + str(iteration) + " | Time : " + str(np.float64(end-start)))


Iteration : 61 | Time : 7.311421155929565
